O objetivo do código aqui é realizar manipulações nos dados
com o objetivo de torná-los estacionários.

Manipulações realizadas:

1. $\Delta x_t$

2. $\Delta^2 x_t$

3. $ln(x_t)$

4. $\Delta ln(x_t)$

5. $\Delta^2 ln(x_t)$

In [2]:
import pandas as pd
import numpy as np
import utils

In [3]:
df = pd.read_csv("dados/dados_crus2003.csv", sep=";", decimal=",", thousands=".")
df.head(5)

,month,ipca,ipca_15,selic,m1,m2,m3,m4,ibovespa,cambio,...,result_primario,el_nino,epu_br,epu_usa,ind_conf_cons,desemprego,rendimento,crb_cmdt,crb_food,crb_metal
0,jan/03,2.25,1.98,25.145161,94977783,391237088,696054502,810272451,10941.09,3.4384,...,48763.52,0.9,88.150920,185.591626,103.0,14.068293,NaN,250.14,239.98,203.34
1,fev/03,1.57,2.19,25.821429,93602937,393976058,706788435,815745074,10280.61,3.5908,...,51066.28,0.6,120.281038,161.759767,103.5,14.570732,NaN,250.13,233.46,206.19
2,mar/03,1.23,1.14,26.500000,89822584,388142457,710712013,820859498,11273.63,3.4469,...,51462.26,0.4,133.688711,224.647203,101.9,15.198780,182585.2573,248.06,234.18,202.43
3,abr/03,0.97,1.14,26.500000,87410599,385467653,712800120,823489095,12556.70,3.1187,...,55840.39,0.0,96.791932,176.579867,107.7,15.575610,178958.1301,247.57,235.03,204.02
4,mai/03,0.61,0.85,26.500000,86231763,388490690,721321334,833412205,13421.60,2.9557,...,57077.43,-0.3,85.230528,122.309831,112.0,16.203659,177284.6602,250.48,238.04,206.54


In [6]:
# Remove colunas de séries que já são estacionárias
# segundo o teste KPSS no arquivo 'stationarity.ipynb'
dropped_df = df.drop(
    columns=[
        'ipca',
        'ipca_15',
        'cambio_real',
        'oil_usa',
        'oil_eu',
        'bens_capital',
        'prod_caminhoes',
        'el_nino',
        'desemprego',
    ]
)
dropped_df.head(5)

,month,selic,m1,m2,m3,m4,ibovespa,cambio,ind_varejo,ind_varejo_ampl,...,prod_veiculos,prod_onibus,result_primario,epu_br,epu_usa,ind_conf_cons,rendimento,crb_cmdt,crb_food,crb_metal
0,jan/03,25.145161,94977783,391237088,696054502,810272451,10941.09,3.4384,16.44619,18.88421,...,132208,1782,48763.52,88.150920,185.591626,103.0,NaN,250.14,239.98,203.34
1,fev/03,25.821429,93602937,393976058,706788435,815745074,10280.61,3.5908,16.13919,18.52597,...,141129,1828,51066.28,120.281038,161.759767,103.5,NaN,250.13,233.46,206.19
2,mar/03,26.500000,89822584,388142457,710712013,820859498,11273.63,3.4469,17.01632,18.83303,...,127486,1857,51462.26,133.688711,224.647203,101.9,182585.2573,248.06,234.18,202.43
3,abr/03,26.500000,87410599,385467653,712800120,823489095,12556.70,3.1187,17.41103,19.39598,...,130783,2019,55840.39,96.791932,176.579867,107.7,178958.1301,247.57,235.03,204.02
4,mai/03,26.500000,86231763,388490690,721321334,833412205,13421.60,2.9557,18.11273,20.21480,...,141857,2146,57077.43,85.230528,122.309831,112.0,177284.6602,250.48,238.04,206.54


In [25]:
# Transformação 1
f = lambda df, i, col_name: df.loc[i, col_name] - df.loc[i - 1, col_name]
delta_df = utils.transform_dataframe(dropped_df, f)
# Ignora meses anteriores a out/2003, pois a previsão começa em 2004
# e o maior lag vai ser 3
# Isso vai se repetir nos próximos arquivos pelo mesmo motivo
delta_df = delta_df.drop(['fev/03', 'mar/03', 'abr/03', 'mai/03', 'jun/03', 'jul/03', 'ago/03', 'set/03'])
delta_df.to_csv('dados/delta.csv', sep=";", decimal=",", index_label='month')

In [26]:
# Transformação 2
f = lambda df, i, col_name: (df.loc[i, col_name] - df.loc[i - 1, col_name]) - (df.loc[i - 1, col_name] - df.loc[i - 2, col_name])
delta_quadrado_df = utils.transform_dataframe(dropped_df, f, skip=[0, 1])
delta_quadrado_df = delta_quadrado_df.drop(['mar/03', 'abr/03', 'mai/03', 'jun/03', 'jul/03', 'ago/03', 'set/03'])
delta_quadrado_df.to_csv('dados/delta_quadrado.csv', sep=";", decimal=",", index_label='month')

In [27]:
# Transformação 3
f = lambda df, i, col_name: np.log(df.loc[i, col_name])
ln_df = utils.transform_dataframe(dropped_df, f, skip=[0, 1])
ln_df = ln_df.drop(['mar/03', 'abr/03', 'mai/03', 'jun/03', 'jul/03', 'ago/03', 'set/03'])
ln_df.to_csv('dados/log.csv', sep=";", decimal=",", index_label='month')

C:\Users\guisf\AppData\Local\Temp\ipykernel_15388\678695276.py:2: RuntimeWarning: invalid value encountered in log
  f = lambda df, i, col_name: np.log(df.loc[i, col_name])


In [29]:
# Tranformação 4
f = lambda df, i, col_name: np.log(df.loc[i, col_name]) - np.log(df.loc[i - 1, col_name])
ln_delta_df = utils.transform_dataframe(dropped_df, f, skip=[0, 1])
ln_delta_df = ln_delta_df.drop(['mar/03', 'abr/03', 'mai/03', 'jun/03', 'jul/03', 'ago/03', 'set/03'])
ln_delta_df.to_csv('dados/delta_log.csv', sep=";", decimal=",", index_label='month')

C:\Users\guisf\AppData\Local\Temp\ipykernel_15388\3021156352.py:2: RuntimeWarning: invalid value encountered in log
  f = lambda df, i, col_name: np.log(df.loc[i, col_name]) - np.log(df.loc[i - 1, col_name])


In [31]:
# Tranformação 5
f = lambda df, i, col_name: (np.log(df.loc[i, col_name]) - np.log(df.loc[i - 1, col_name])) - (np.log(df.loc[i - 1, col_name]) - np.log(df.loc[i - 2, col_name]))
ln_delta_quadrado_df = utils.transform_dataframe(dropped_df, f, skip=[0, 1])
ln_delta_quadrado_df = ln_delta_quadrado_df.drop(['mar/03', 'abr/03', 'mai/03', 'jun/03', 'jul/03', 'ago/03', 'set/03'])
ln_delta_quadrado_df.to_csv('dados/delta_log_quadrado.csv', sep=";", decimal=",", index_label='month')

C:\Users\guisf\AppData\Local\Temp\ipykernel_15388\3961766518.py:2: RuntimeWarning: invalid value encountered in log
  f = lambda df, i, col_name: (np.log(df.loc[i, col_name]) - np.log(df.loc[i - 1, col_name])) - (np.log(df.loc[i - 1, col_name]) - np.log(df.loc[i - 2, col_name]))


O código gera corretamente as versões transformadas,
mas fizeram-se algumas mudanças manuais nos arquivos posteriormente.

Removeram-se as entradas anteriores a out/2003, pois o maior lag que vai se utilizar é de 3.